In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('heart_disease.csv')  # 파일명은 환경에 맞게 수정

# 3. 특성(X)과 레이블(y) 분리
X = df.loc[:, "age":"thal"]
y = df[['target']]

# 4. 특성 데이터 평균 및 표준편차 계산
X_mean = X.mean()
X_std = X.std()

# 5. 특성 데이터 정규화
X_norm = (X - X_mean) / X_std

# 6. 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [4]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [14]:
X_test_tensor

tensor([[62.,  0.,  0.,  ...,  2.,  0.,  2.],
        [53.,  0.,  2.,  ...,  2.,  0.,  0.],
        [55.,  1.,  0.,  ...,  1.,  1.,  3.],
        ...,
        [44.,  1.,  2.,  ...,  2.,  0.,  2.],
        [61.,  1.,  0.,  ...,  2.,  1.,  3.],
        [58.,  1.,  2.,  ...,  1.,  1.,  3.]])

In [15]:
input_dim = X_train.shape[1]
input_dim

13

In [6]:
linear = torch.nn.Linear(input_dim, 1)

In [7]:
optimizer = optim.Adam(linear.parameters(), lr=0.01)
loss_fn = torch.nn.BCELoss()

In [17]:
epochs = 1000
for epoch in range(epochs):
    z = linear(X_train_tensor)
    y_pred = torch.sigmoid(z)
    loss = loss_fn(y_pred, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch+1) % 100 == 0 or epoch == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

Epoch 1/1000, Loss: 48.4146
Epoch 100/1000, Loss: 48.4146
Epoch 200/1000, Loss: 48.4146
Epoch 300/1000, Loss: 48.4146
Epoch 400/1000, Loss: 48.4146
Epoch 500/1000, Loss: 48.4146
Epoch 600/1000, Loss: 48.4146
Epoch 700/1000, Loss: 48.4146
Epoch 800/1000, Loss: 48.4146
Epoch 900/1000, Loss: 48.4146
Epoch 1000/1000, Loss: 48.4146


In [9]:
with torch.no_grad():
    z_test = linear(X_test_tensor)
    y_test_pred = torch.sigmoid(z_test)
    y_test_pred_label = (y_test_pred >= 0.5).float()
    accuracy = (y_test_pred_label == y_test_tensor).float().mean()
    print(f"\n테스트 정확도: {accuracy.item() * 100:.2f}%")


테스트 정확도: 50.24%


In [10]:
new_patient_1 = pd.DataFrame({
    'age': [59],
    'sex': [0],
    'cp': [0],
    'trestbps': [98],
    'chol': [253],
    'fbs': [0],
    'restecg': [0],
    'thalach': [119],
    'exang': [0],
    'oldpeak': [1.0],
    'slope': [1],
    'ca': [0],
    'thal': [2]
})

In [ ]:
with torch.no_grad():
    new_patient_1_norm = (new_patient_1 - X_mean) / X_std
    patient1_tensor = torch.tensor(new_patient_1_norm.values, dtype=torch.float32)
    pred1 = torch.sigmoid(linear(patient1_tensor))
    print(f"\n환자 1의 심장병 예측 확률: {pred1.item():.4f}")